In [1]:
import numpy as np

enviroment_rows = 11
enviroment_columns = 11

q_values = np.zeros((enviroment_rows, enviroment_columns, 4))

# definir acciones como posiciones del arreglo
actions = ['up', 'right', 'down', 'left']

In [2]:
# Recompensas en general, incluyendo estantes para que no tome esa opción
rewards = np.full((enviroment_rows, enviroment_columns), -100.)

# ubicacions de aisles
aisles = {}
aisles[1] = [i for i in range(1, 10)]
aisles[2] = [1, 7, 9]
aisles[3] = [i for i in range(1, 8)]
aisles[3].append(9)
aisles[4] = [3, 7]
aisles[5] = [i for i in range(11)]
aisles[6] = [5]
aisles[7] = [i for i in range(1, 10)]
aisles[8] = [3, 7]
aisles[9] = [i for i in range(11)]

# Asignar recompensa en los pasillos o donde si puede pasar
for row_index in range(1, 10):
  for column_index in aisles[row_index]:
    rewards[row_index, column_index] = -1.

for row in rewards:
  print(row)

[-100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100.]
[-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.]
[-100.   -1. -100. -100. -100. -100. -100.   -1. -100.   -1. -100.]
[-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.   -1. -100.]
[-100. -100. -100.   -1. -100. -100. -100.   -1. -100. -100. -100.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[-100. -100. -100. -100. -100.   -1. -100. -100. -100. -100. -100.]
[-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.]
[-100. -100. -100.   -1. -100. -100. -100.   -1. -100. -100. -100.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[-100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100.]


In [3]:
# Funciones de apoyo
def is_terminal_state(current_row_index, current_column_index):
  if rewards[current_row_index, current_column_index] == -1.:
    return False
  else:
    return True

def get_starting_location():
  current_row_index = np.random.randint(enviroment_rows)
  current_column_index = np.random.randint(enviroment_columns)

  while is_terminal_state(current_row_index, current_column_index):
    current_row_index = np.random.randint(enviroment_rows)
    current_column_index = np.random.randint(enviroment_columns)
  
  return current_row_index, current_column_index

# Epsilon greedy alg
def get_next_action(current_row_index, current_column_index, epsilon):
  if np.random.random() < epsilon:
    return np.argmax(q_values[current_row_index, current_column_index])
  else:
    return np.random.randint(4)

In [4]:
def get_next_location(current_row_index, current_column_index, action_index):
  new_row_index = current_row_index
  new_column_index = current_column_index

  if actions[action_index] == 'up' and current_row_index > 0:
    new_row_index -= 1
  elif actions[action_index] == 'right' and current_column_index < enviroment_columns - 1:
    new_column_index += 1
  elif actions[action_index] == 'down' and current_row_index < enviroment_rows - 1:
    new_row_index += 1
  elif actions[action_index] == 'left' and current_column_index > 0:
    new_column_index -= 1

  return new_row_index, new_column_index

def get_shortest_path(start_row_index, start_column_index):
  # Retorno inmediato si inicia en una posición inválida
  if is_terminal_state(start_row_index, start_column_index):
    return []
  else:
    current_row_index, current_column_index = start_row_index, start_column_index
    shortest_path = []
    shortest_path.append([start_row_index, start_column_index])

    while not is_terminal_state(current_row_index, current_column_index):
      # Obtener el mejor movimiento
      action_index = get_next_action(current_row_index, current_column_index, 1.)

      # moverse a la siguiente posición y añadirla a la lista de movimientos
      current_row_index, current_column_index = get_next_location(current_row_index, current_column_index, action_index)
      shortest_path.append([current_row_index, current_column_index])
    
    return shortest_path

In [5]:
epsilon = 0.9 # Porcentaje de tiempo en que debemos tomar la mejor decisión
discount_factor = 0.9 # Descuento para futuras recompensas
learning_rate = 0.9

for episode in range(1000):
  row_index, column_index = get_starting_location()

  while not is_terminal_state(row_index, column_index):
    action_index = get_next_action(row_index, column_index, epsilon)

    old_row_index, old_column_index = row_index, column_index
    row_index, column_index = get_next_location(row_index, column_index, action_index)

    reward = rewards[row_index, column_index]
    old_q_value = q_values[old_row_index, old_column_index, action_index]
    temporal_difference = reward + (discount_factor * np.max(q_values[row_index, column_index])) - old_q_value

    new_q_value = old_q_value + (learning_rate * temporal_difference)
    q_values[old_row_index, old_column_index, action_index] = new_q_value

print('training complete')

training complete


In [6]:
print(get_shortest_path(3, 9))
print(get_shortest_path(5, 0))
print(get_shortest_path(9, 5))